In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re
import datetime

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
# from vivarium import Artifact

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output.measures import VPHResults
from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer
# from vivarium_helpers.vph_output import cleaning
# from vivarium_helpers.vph_output.loading import load_draws_from_keyspace_files
from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
# from vivarium_helpers.vph_artifact.operations import convert_to_sim_format
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

In [2]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [3]:
!ls $output_dir.parent

results_10_31_2025	 results_11_14_2025_mslt
results_10_31_2025_mslt  results_in_progress


In [12]:
!ls $output_dir.parent/results_11_14_2025_mslt

2025_11_14__14_11_42_outside_sim_results_draw_level.csv


In [16]:
mslt_draw_data = pd.read_csv(
    output_dir.parent / 'results_11_14_2025_mslt/2025_11_14__14_11_42_outside_sim_results_draw_level.csv',
    index_col=0
    )
mslt_draw_data

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Draw,Value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [4]:
!ls $output_dir

bbbm_tests.csv	   dalys.csv   incidence.csv   prevalence.csv
csf_pet_tests.csv  deaths.csv  medication.csv  version1


In [5]:
bbbm_tests_new = pd.read_csv(output_dir / 'bbbm_tests.csv')
bbbm_tests_new

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
121595,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Positive BBBM Tests,Number,310309.913080,254337.601734,343534.790966
121596,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.043111,0.038218,0.046166
121597,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.047412,0.042346,0.050779
121598,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing and Treatment,Positive BBBM Tests,Rate,0.043111,0.038218,0.046166


In [6]:
bbbm_tests_old = pd.read_csv(output_dir / 'version1/bbbm_tests.csv')
bbbm_tests_old

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,25_to_29,Female,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,25_to_29,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,25_to_29,Male,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,25_to_29,Male,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
91815,2100,United States of America,75_to_80,Female,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Number,146774.863478,136236.289374,154975.735010
91816,2100,United States of America,60_to_65,Male,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Number,231725.331175,215371.159718,244914.562719
91817,2100,United States of America,65_to_70,Male,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Number,187697.518252,174450.639371,198380.795803
91818,2100,United States of America,70_to_75,Male,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Number,80018.415676,74371.062048,84572.865579


In [9]:
bbbm_tests_old.Age.unique()

array(['25_to_29', '30_to_34', '35_to_39', '40_to_44', '45_to_49',
       '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74',
       '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus',
       '60_to_65', '65_to_70', '70_to_75', '75_to_80'], dtype=object)

In [7]:
bbbm_tests_new.query("Metric=='Number'")

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Tests,Number,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
121587,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,BBBM Tests,Number,3.103099e+06,2.543376e+06,3.435348e+06
121592,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing Only,Positive BBBM Tests,Number,3.328697e+05,2.724603e+05,3.693527e+05
121593,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Number,3.103099e+05,2.543376e+05,3.435348e+05
121594,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing and Treatment,Positive BBBM Tests,Number,3.328697e+05,2.724603e+05,3.693527e+05


In [11]:
bbbm_tests_new.query(
    "Year == 2100 & Location == 'United States of America' & Age == '75_to_79' & Sex == 'Male' & `Disease Stage` == 'Susceptible' & Scenario == 'BBBM Testing Only' & Measure == 'Positive BBBM Tests'"
)

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
121593,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Number,310309.913080,254337.601734,343534.790966
121597,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.047412,0.042346,0.050779


In [17]:
mslt_draw_data.query(
    "Year == 2100 & Location == 'United States of America' & Age == '75_to_79' & Sex == 'Male' & `Disease Stage` == 'Susceptible' & Scenario == 'BBBM Testing Only' & Measure == 'BBBM False Positive Tests'"
)

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Draw,Value
151975,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,113,129492.117922
151976,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,13,135500.555788
151977,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,158,128735.763960
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,169,141981.696594
151979,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,177,137260.741308
151980,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,199,141810.697611
151981,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,2,139997.318809
151982,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,24,139208.310909
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,258,135267.959115
151984,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM False Positive Tests,Number,262,139145.555116


In [19]:
_17['Value'].mean()

136831.4908054301

In [20]:
_*2

273662.9816108602

In [21]:
310/136

2.2794117647058822

In [22]:
136/310

0.43870967741935485